### Simple GEN_AI App With Langchain

In [26]:
import os
import dotenv
dotenv.load_dotenv()

os.environ["GOOGLE_GENAI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [ ]:
# Document Creation
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.geeksforgeeks.org/what-is-generative-ai/")
pages = loader.load()

[Document(metadata={'source': 'https://www.geeksforgeeks.org/what-is-generative-ai/', 'title': 'What is Generative AI? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhat is Generative AI? - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPracticePractice Coding 

In [ ]:
# Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 2500,
  chunk_overlap = 200,
)
documents = text_splitter.split_documents(pages)
documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/what-is-generative-ai/', 'title': 'What is Generative AI? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="What is Generative AI? - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPracticePractice Coding ProblemsNation Skillup- Free CoursesProblem of the DayJobsApply Now!Post JobsJobs Updates\n\n\n\n\

In [ ]:
# Embedding and Vector Store
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectorStore = FAISS.from_documents(documents, embeddings)

C:\Users\User\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [50]:
# Querying
query = " type of artificial intelligence designed to create new content."
results = vectorStore.similarity_search(query)
results[0].page_content

'Generative AI is a type of artificial intelligence designed to create new content such as text, images, music or even code by learning patterns from existing data. These models generate original outputs that are often indistinguishable from human-created content. These models use techniques like deep learning and neural networks to generate output.Unlike discriminative AI which focuses on classifying data into categories like spam vs. not spam, generative AI creates new data such as text, images, audio or video that resembles real-world examples.How Generative AI Works1. Core Mechanism (Training & Inference)Generative AI is trained on large datasets like text, images, audio or video using deep learning networks. During training, the model learns parameters (millions or billions of them) that help them predict or generate content. Here models generate output based on learned patterns and prompts provided2. By Media TypeText: Uses large language models (LLMs) to predict the next token i

In [55]:
# Retrival Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
  model="openai/gpt-oss-20b",
  api_key=os.getenv("GROQ_API_KEY"),
  temperature=0,
  max_tokens=1024
)

prompt = ChatPromptTemplate.from_template(
  """
  Answer the question based on the provided context:
  <context>
  {context}
  </context>
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain.invoke({
  "input": "Scopes of Generative AI?",
  "context": documents
})

'**Generative AI** is a branch of artificial intelligence that learns patterns from large datasets and then uses that knowledge to create brand‑new content—text, images, audio, video, code, or other data types. Unlike discriminative models that simply classify or predict labels, generative models generate original outputs that can be indistinguishable from human‑made content.  \n\nKey points from the GeeksforGeeks article:\n\n| Feature | Description |\n|---------|-------------|\n| **Purpose** | Produce new, creative content (e.g., chat responses, artwork, music, code). |\n| **Core Mechanisms** | • **Transformers / Autoregressive models** (e.g., GPT) – predict next token step‑by‑step. <br>• **Diffusion models** – start from noise and iteratively denoise to form images or audio. <br>• **GANs & VAEs** – adversarial or probabilistic latent‑space generation. <br>• **Encoder‑Decoder architectures** – translate or summarize by encoding input and decoding output. |\n| **Training** | Large‑scal